## Optimización de parámetros para Arboles de Decisión


In [ ]:
import pandas as pd

dataset_work_3 = pd.read_csv('D:\DataSet\dataset_work_transformed.csv')
x = dataset_work_3.iloc[:,:-1]
y = dataset_work_3.iloc[:,-1].values
dataset_work_3.shape

(6435823, 19)

In [2]:
import pandas as pd
import numpy as np
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Registro del tiempo de inicio
start_time = time.time()

# 2. Separar las características (X) y la variable objetivo (y)
X = dataset_work_3.drop(columns=["MOROSIDAD"])
y = dataset_work_3["MOROSIDAD"]

# 3. Escalado de características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 5. Espacio de búsqueda para los hiperparámetros
param_dist = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

# 6. Inicializar el clasificador y RandomizedSearchCV
clf = DecisionTreeClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=10,  # Reducido para acelerar la búsqueda
    cv=2,  # Validación cruzada con 2 folds
    n_jobs=-1,  # Usar múltiples núcleos
    verbose=2,
    random_state=42,
    scoring='roc_auc'
)

# 7. Ajustar el modelo con la muestra del dataset
random_search.fit(X_train, y_train)

# 8. Obtener los mejores parámetros
print("Mejores parámetros encontrados:")
print(random_search.best_params_)

# 9. Entrenar el modelo final con todos los datos utilizando los mejores parámetros
best_params = random_search.best_params_
best_model = DecisionTreeClassifier(**best_params, random_state=42)

# Ajustar el modelo con el dataset completo
X_full = dataset_work_3.drop(columns=["MOROSIDAD"])
y_full = dataset_work_3["MOROSIDAD"]
X_full_scaled = scaler.transform(X_full)

# División de los datos para el modelo final
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full_scaled, y_full, test_size=0.3, random_state=42)

# Ajustar el modelo con el dataset completo
best_model.fit(X_train_full, y_train_full)

# Hacer predicciones y evaluar el modelo
y_pred = best_model.predict(X_test_full)
y_pred_proba = best_model.predict_proba(X_test_full)[:, 1]

# Métricas de evaluación
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test_full, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test_full, y_pred))

roc_auc = roc_auc_score(y_test_full, y_pred_proba)
print(f"ROC AUC: {roc_auc:.2f}")

# Medir el tiempo de finalización y calcular el tiempo transcurrido
end_time = time.time()
elapsed_time = end_time - start_time

# Convertir a minutos y segundos
minutes = elapsed_time // 60
seconds = elapsed_time % 60

print(f"El tiempo total de ejecución fue de {int(minutes)} minutos y {seconds:.2f} segundos.")



Fitting 2 folds for each of 10 candidates, totalling 20 fits


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
4 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/conda/lib/python3.10/site-packages/sklearn/utils/_pa

Mejores parámetros encontrados:
{'splitter': 'best', 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 10, 'criterion': 'gini'}

Matriz de Confusión:
[[913022 207817]
 [190392 619516]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.83      0.81      0.82   1120839
           1       0.75      0.76      0.76    809908

    accuracy                           0.79   1930747
   macro avg       0.79      0.79      0.79   1930747
weighted avg       0.79      0.79      0.79   1930747

ROC AUC: 0.90
El tiempo total de ejecución fue de 4 minutos y 19.48 segundos.
